In [1]:
def adjust_gamma(image, gamma = 1.0):
    #build a lookup table mapping the pixel values [0. 255] to their adjusted
    #gamma values. 
    invGamma = 1.0/ gamma
    table = np.array([((i/255.0)** invGamma) *255
        for i in np.arange(0, 256)]).astype("uint8")
    #apply gamma correction using the lookup table. 
    return cv2.LUT(image, table)


In [2]:
import numpy as np
import cv2

img = cv2.imread('/Users/karishmamuni/Desktop/Underexposed.jpg')

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
cv2.imshow('image', gray)

gamma = 0.5
adjusted = adjust_gamma(gray, gamma=gamma)
cv2.putText(adjusted, "g={}".format(gamma), (10, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 3)
cv2.imshow("gammam image 1", adjusted)

cv2.waitKey(0)
cv2.destroyAllWindows()

error: OpenCV(4.1.1) /Users/travis/build/skvark/opencv-python/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [3]:
import numpy as np
import cv2

img = cv2.imread('/Users/karishmamuni/Desktop/Overexposed.jpg')

grayoverexposed = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
cv2.imshow('overexposedimage', grayoverexposed)

gamma = 0.5
adjusted = adjust_gamma(gray, gamma=gamma)
cv2.putText(adjusted, "g={}".format(gamma), (10, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 3)
cv2.imshow("gammam image 2", adjusted)

cv2.waitKey(0)
cv2.destroyAllWindows()

error: OpenCV(4.1.1) /Users/travis/build/skvark/opencv-python/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
